# Clasificador de funciones con eliminacion recursiva (RFE)

Es una tecnica utilizada para seleccionar las caracteristicas mas importantes de un conjunto de datos. Su objetivo es eliminar las caracteristicas menos relevantes, mejorando asi el rendimiento del modelo y reduciendo su complejidad.

Generalmente, RFE se aplica a modelos que pueden proporcionar alguna medida de la importancia de las caracteristicas como los modelos lineales o arboles de decision. Entre los modelos donde se puede aplicar tenemos:

- Regresion Lineal
- Regresion Logistica
- Arboles de decision
- Bosques aleatorios
- Maquina de vectores de soporte
- Entre otros...

# Como funciona RFE?

   1. **Entrenamiento inicial**: Entrena un modelo utilizando todas las caracteristicas (Variables predictoras)
    
    
   2. **Evaluacion de Importancia**: Evalua la importancia de cada caracteristica. Esta importancia se puede medir de diferentes maneras dependiendo del modelo utilizado (coeficientes de la regresion lineal)
   
    
   3. **Eliminacion de Caracteristicas**: Elimina las caracteristicas menos importantes para el modelo
   
   
   4. **Repeticion de pasos**: Repite los pasos de 1 a 3 hasta alcanzar un numero predefinido de caracteristicas.

# Aplicacion de RFE sobre un modelo de Regresion Lineal

Supongamos que tenemos un conjunto de 5 caracteristicas y vamos a usar RFE para seleccionar las 3 caracteristicas mas importantes. Generamos el conjunto de datos de prueba

In [1]:
# Generamos datos de prueba
import numpy as np
import pandas as pd

# semilla
np.random.seed(0)

# 5 caracteristicas
X = pd.DataFrame({
    'feature_1': np.random.rand(100),
    'feature_2': np.random.rand(100),
    'feature_3': np.random.rand(100),
    'feature_4': np.random.rand(100),
    'feature_5': np.random.rand(100)
})

# variable objetivo
# la asociamos linealmente con 3 de las 5 caracteristicas
y = 3*X['feature_2'] + 2*X['feature_5'] + X['feature_1'] + np.random.rand(100)

In [3]:
# juntamos los datos en un dataframe para mejorar la visualizacion de los datos
df = pd.DataFrame(X, columns=[f'feature_{i+1}' for i in range(X.shape[1])])
df['target'] = y
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,target
0,0.548814,0.677817,0.311796,0.906555,0.401260,3.695163
1,0.715189,0.270008,0.696343,0.774047,0.929291,3.756831
2,0.602763,0.735194,0.377752,0.333145,0.099615,3.532546
3,0.544883,0.962189,0.179604,0.081101,0.945302,6.072647
4,0.423655,0.248753,0.024679,0.407241,0.869489,3.242399


### 1.- Entrenamiento inicial
Entrenamos el modelo de regresion lineal con todas las caracteristicas y calculamos la importancia de cada una.

In [7]:
# libreria para usar la regresion lineal
from sklearn.linear_model import LinearRegression

# funcion para entrena el modelo
def train_model(X, y):
    model = LinearRegression()
    return model.fit(X,y)

In [8]:
# entrenamos el modelo
modelo = train_model(X, y)

### 2.- Evaluacion de importancia
Para el caso de la regresion, debemos obtener los coeficientes de cada caracteristicas una vez entrenado el modelo.

In [9]:
# funcion para obtener los coeficientes de cada caracteristica
def evaluate_model(model):
    coeficientes = model.coef_
    return coeficientes

In [10]:
# coeficiente de las caracteristicas
coefi = evaluate_model(modelo)
coefi

array([1.0169788 , 2.96313024, 0.01281724, 0.04742558, 1.94336239])

### 3.- Eliminacion de caracteristicas
Identificamos la caracteristica con el coeficiente mas pequeño (el valor absoluto) y la eliminamos

In [11]:
# funcion que determina la caracteristica con menor valor de coeficiente y la elimina
def remove_feature(X, coeficientes):
    # indice del coficiente mas pequeño
    min_indice = np.argmin(np.abs(coeficientes))
    # nombre de la caracteristica correspondiente al indice anterior
    feature_remove = X.columns[min_indice]
    # eliminar caracteristica
    X = X.drop(columns=[feature_remove])
    # retornar matriz de caracteristica y nombre de la caracteristica eliminada
    return X, feature_remove
    

In [12]:
# eliminamos la caracteristica menos importante
X, feature_remove = remove_feature(X, coefi)
print(feature_remove)
print(X.columns)

feature_3
Index(['feature_1', 'feature_2', 'feature_4', 'feature_5'], dtype='object')
